<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from catboost import CatBoostClassifier
import nltk
nltk.download('averaged_perceptron_tagger')
import spacy
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer 
import time
sns.set(style='darkgrid', font_scale=1.4)

In [ ]:
try:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv('datasets/toxic_comments.csv', index_col=0) 

In [ ]:
data.head()

In [ ]:
data.duplicated().sum()

In [ ]:
data['toxic'].value_counts().plot.pie(explode=[0.1,0.1], autopct='%1.1f%%', shadow=True,
                                             textprops={'fontsize':16}).set_title("Распределение целевого признака");

Имеем большой дисбаланс классов

In [ ]:
plt.figure(figsize=(15,8))
data['length'] = data['text'].apply(len)
ax = sns.histplot(data=data, x='length', hue='toxic', kde=True)
plt.title('Распределение количества слов в коментарии')
plt.ylabel('Количество коментариев')
plt.xlabel('Количество слов');

Распределение длин коментариев похоже, отличается только их масштаб

In [ ]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
def lemmatize(text):
    m = WordNetLemmatizer()
    words = text.split()
    words = [m.lemmatize(word.lower(), get_wordnet_pos(word)) for word in words]
    return ' '.join(words)    

def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z]', ' ', text).split()) 

In [ ]:
%%time
data['text_lemm'] = data['text'].apply(lambda x: lemmatize(clear_text(x)))

In [ ]:
data.head()

In [ ]:
X=data['text_lemm']
y=data['toxic']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size= 0.25, random_state= 42)

## Обучение

In [ ]:
models = {
    #"LGBM" : LGBMClassifier(random_state=42),
    "Ridge": RidgeClassifier(random_state=42),
    "NB": MultinomialNB(), 
    "SVC": LinearSVC(),
    "LogisticRegression" : LogisticRegression(random_state=42),
    "RandomForest" : RandomForestClassifier(random_state=42)
}
LR_grid = {'model__C': [1e-5, 1e-3, 1e-1, 1e0, 1e1],
          'model__solver': ['sag'],
           'model__multi_class': ['multinomial'],
          'model__max_iter': [10000],
          'model__penalty': ['l2']}

NB_grid = {'model__alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}  

KNN_grid = {'model__n_neighbors': [3, 5, 7, 9],
            'model__p': [1, 2]}

SVC_grid = {'model__C': [0.25, 0.5, 0.75, 1, 1.25, 1.5]}

RF_grid = {'model__n_estimators': [50, 80, 100],
        'model__max_depth': [4, 6, 8, 10, 12]}

boosted_grid = {
        'model__max_depth': [4, 6, 8, 10],
        'model__learning_rate': [0.05, 0.1, 0.15]}
Ridge_grid = {'model__alpha':[200, 230, 250, 270, 290, 300]}
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    "SVC" : SVC_grid,
    "RandomForest" : RF_grid,
    "LGBM" : boosted_grid,
    "CatBoost" : boosted_grid,
    "Ridge": Ridge_grid,
    "NB": NB_grid
}

In [ ]:
def pipeline(X_train, y_train):
    i=0
    model_best_params=models.copy()
    model_best_estimator=models.copy()
    valid_scores=pd.DataFrame({'Model':models.keys(), 'F1': np.zeros(len(models)), 'Training time': np.zeros(len(models))})
    for key, model in models.items():
        pipeline = Pipeline([( "vectorizer" , TfidfVectorizer(stop_words=set(nltk_stopwords.words('english'))
                                                          , max_features=10000, analyzer='word'))
                             ,('model', model)])
        model = RandomizedSearchCV(estimator=pipeline, param_distributions=grid[key], n_jobs=-1,
                                 cv=3, scoring='f1', verbose=2)
        start = time.time()

        model.fit(X_train, y_train)
        valid_scores.iloc[i,1]=model.best_score_

        model_best_params[key]=model.best_params_
        model_best_estimator[key]=model.best_estimator_ 
        
        stop = time.time()
        valid_scores.iloc[i,2]=np.round((stop - start)/60, 2)

        print('Model:', key)
        print('Training time (mins):', valid_scores.iloc[i,2])
        print('')
        i+=1
    return valid_scores, model_best_params, model_best_estimator  

In [ ]:
valid_scores, best_params, best_estimator = pipeline(X_train, y_train)

In [ ]:
valid_scores

In [ ]:
best_params

На валидации лучшее качество у модели LinearSVC

## Выводы

In [ ]:
stopwords = set(nltk_stopwords.words('english'))
tfidf=TfidfVectorizer(stop_words=stopwords, max_features=10000, analyzer='word',)
X_train=tfidf.fit_transform(X_train)
X_test=tfidf.transform(X_test)

In [ ]:
final_model = LinearSVC(C=1, random_state=42)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
print('F1:', f1_score(y_test, y_pred))

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
ax = sns.heatmap(cf_matrix, annot=True, fmt='d', )
ax.set_xlabel("Прогоз", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(['Negative', 'Positive'])
ax.set_ylabel("Реальность", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(['Negative', 'Positive'])
ax.set_title("Матрица ошибок", fontsize=14, pad=20)
plt.show()

In [ ]:
dummy_cl = DummyClassifier(strategy="constant", constant=1) 
dummy_cl.fit(X_train, y_train) 
dummy_pred = dummy_cl.predict(X_test)
print("F1:", f1_score(dummy_pred, y_test))

In [ ]:
cf_matrix = confusion_matrix(y_test, dummy_pred)
plt.figure(figsize=(6,4))
ax = sns.heatmap(cf_matrix, annot=True, fmt='d', )
ax.set_xlabel("Прогоз", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(['Negative', 'Positive'])
ax.set_ylabel("Реальность", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(['Negative', 'Positive'])
ax.set_title("Матрица ошибок", fontsize=14, pad=20)
plt.show()

## Вывод

Для построения модели классифицикации комментариев на позитивные и негативные, корпус текстов был очищен от посторонних символов и преведен к лемме. Полученные тексты были векторизированы. В распределении таргета имеется большой дисбаланс классов. На кросс-валидации выбрана модель LinearSVC, показавшая наилучшее качество. Значение метрики F1 сотавило 0.78, что выше заданного бейзлайна. 